In [95]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
from nltk.corpus import stopwords 
import re
analyser = SentimentIntensityAnalyzer()
import pyodbc

In [161]:
def connection():
    DRIVER_NAME = 'SQL Server'
    s = 'DESKTOP-JBUBG3B' #Your server name 
    d = 'Fashion_Trends_2022' 
    u = "DESKTOP-JBUBG3B" #Your login
    p = ' ' #Your login password
    conn_string = f"""
        Driver={{{DRIVER_NAME}}};
        Server={s};
        Database={d};
        Trust_Connection=yes;"""
    conn = pyodbc.connect(conn_string)
    return conn

conn = connection()
cursor = conn.cursor()
cursor.execute('''select * from Product_Details''')
record = cursor.fetchall()
data=pd.DataFrame(record)

# data=pd.read_csv("dress-flipkart-final-final.csv")
data.head()
# datas=['URL','ID','ITEM','STARS','NUMBER OF RATINGS','NUMBER OF REVIEWS', 'LIST OF REVIEWS']
# datas=data.reindex(datas)
# data.head()
# data=data[['URL','ID','ITEM','STARS','NUMBER OF RATINGS','NUMBER OF REVIEWS', 'LIST OF REVIEWS']]

""


In [97]:
data.rename(columns = {'ID':'id','ITEM':'desc', 'LIST OF REVIEWS':'reviews','NUMBER OF REVIEWS':'num_reviews','STARS':'stars','NUMBER OF RATINGS':'num_ratings'}, inplace=True)
data.head()

,Unnamed: 0,URL,id,desc,stars,num_ratings,num_reviews,reviews,vader_score,final_score
0,0,http://flipkart.com/aayu-women-fit-flare-multi...,0,Women Fit and Flare Multicolor Dress,4.0,4510,867,nice🙂👍 nice fabric beautiful dress perfect for...,3.1971,0.045085
1,1,http://flipkart.com/micozy-women-a-line-multic...,1,Women A-line Multicolor Dress,4.0,12612,2026,very very nice dress 💃💃💃💃💃 good good,1.3023,0.087918
2,2,http://flipkart.com/daevish-women-a-line-black...,2,Women A-line Black Dress,4.3,6532,1445,i love this dress 👌👌 good quality perfect dress,2.5547,0.065537
3,3,http://flipkart.com/limeswood-creation-women-m...,3,"Women Maxi Black, Yellow Dress",3.9,408,50,good product nice dress but finishing was not ...,1.1351,0.002576
4,4,http://flipkart.com/tokyo-talkies-women-maxi-b...,4,Women Maxi Black Dress,4.2,1173,283,outfit is js osam great dress very verry nice...,2.5140,0.011973


In [98]:
data.reviews = data.reviews.str.lower()
data.reviews = data.reviews.str.replace('\n','').str.replace('[\'!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]','')

C:\Users\abhay\AppData\Local\Temp\ipykernel_8032\2430931611.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data.reviews = data.reviews.str.replace('\n','').str.replace('[\'!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]','')


In [99]:
data['vader_score'] = 1.0
for ind in data.index: 
    rev=data['reviews'][ind]
    print(rev)
    x = str(rev).split()
#     print(x)
    sum_score=0.0
    for i in x:
        score = analyser.polarity_scores(i)
        sum_score=sum_score+score['compound']
    data['vader_score'][ind] =sum_score

C:\Users\abhay\AppData\Local\Temp\ipykernel_8032\680855166.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['vader_score'][ind] =sum_score


nice🙂👍 nice fabric beautiful dress perfect for feeding as all buttons are openable belt is also provided thank u flipkart excellent product
very very nice dress 💃💃💃💃💃 good good
i love this dress 👌👌 good quality perfect dress
good product nice dress but finishing was not well nyc product
outfit is js osam great dress very verry  nice😍😍😍go for itim  very happy with  this purchase 😘
very good qualityso happy with this very good good product i am happy with this product 😊😊😊😊 nice product i love this
this dress is so good and beautiful i just love it  and it is same as shown very nice product colour is not good
good product good product  colour material everything is nice nice product
just wow go for it best product nice
ausome too good osm product love this 😍 very good product
very beautiful dress i love this dress i am so happy 😇😇😇😍 excellent brought it for my sis she loves it 😍
very nice kurti very good quality 100 cotten color same as show in picture but it little bit longotherwish perf

In [100]:
total_star_givers = data['num_ratings'].sum()     
total_review_givers = data['num_reviews'].sum()
# print(total_star_givers)
# print(total_review_givers)

In [101]:
data['final_score'] = ((data['stars']*data['num_ratings'])/total_star_givers)+((data['vader_score']*data['num_reviews'])/total_review_givers)

In [102]:
data.head()

,Unnamed: 0,URL,id,desc,stars,num_ratings,num_reviews,reviews,vader_score,final_score
0,0,http://flipkart.com/aayu-women-fit-flare-multi...,0,Women Fit and Flare Multicolor Dress,4.0,4510,867,nice🙂👍 nice fabric beautiful dress perfect for...,3.1971,0.045085
1,1,http://flipkart.com/micozy-women-a-line-multic...,1,Women A-line Multicolor Dress,4.0,12612,2026,very very nice dress 💃💃💃💃💃 good good,1.3023,0.087918
2,2,http://flipkart.com/daevish-women-a-line-black...,2,Women A-line Black Dress,4.3,6532,1445,i love this dress 👌👌 good quality perfect dress,2.5547,0.065537
3,3,http://flipkart.com/limeswood-creation-women-m...,3,"Women Maxi Black, Yellow Dress",3.9,408,50,good product nice dress but finishing was not ...,1.1351,0.002576
4,4,http://flipkart.com/tokyo-talkies-women-maxi-b...,4,Women Maxi Black Dress,4.2,1173,283,outfit is js osam great dress very verry nice...,2.5140,0.011973


In [103]:
data.to_csv('dresses_csv_final.csv')

In [104]:
    
dict_bigram={}
item_bigram={}
stop_words = set(stopwords.words('english')) 
for ind in data.index:
    d=data['desc'][ind][6:-6]
    print(d)
    d = re.sub(r'[^\w\s]', '', d) 
    print("Yo",d)
    nltk_tokens = nltk.word_tokenize(d,"english",preserve_line=True)
    nltk_tokens = [w for w in nltk_tokens if not w in stop_words] 
    arr=list(nltk.bigrams(nltk_tokens))
    for i in arr:
        if i in dict_bigram:
            dict_bigram[i]=dict_bigram[i]+1
            item_bigram[" ".join(i)].append(ind)
        else:
            dict_bigram[i]=1
            item_bigram[" ".join(i)]=[]
            item_bigram[" ".join(i)].append(ind)
print(item_bigram) 

Fit and Flare Multicolor
Yo Fit and Flare Multicolor
A-line Multicolor
Yo Aline Multicolor
A-line Black
Yo Aline Black
Maxi Black, Yellow
Yo Maxi Black Yellow
Maxi Black
Yo Maxi Black
Fit and Flare Yellow Dress Wit
Yo Fit and Flare Yellow Dress Wit
Fit and Flare Multicolor
Yo Fit and Flare Multicolor
A-line Multicolor
Yo Aline Multicolor
Skater Multicolor
Yo Skater Multicolor
Fit and Flare Multicolor
Yo Fit and Flare Multicolor
A-line Multicolor
Yo Aline Multicolor
Fit and Flare Multicolor
Yo Fit and Flare Multicolor
Bandage Yellow
Yo Bandage Yellow
Gown Maroon
Yo Gown Maroon
Maxi Maroon
Yo Maxi Maroon
Maxi Blue
Yo Maxi Blue
Bodycon Multicolor
Yo Bodycon Multicolor
Fit and Flare Blue
Yo Fit and Flare Blue
Maxi Yellow Dress Wit
Yo Maxi Yellow Dress Wit
A-line Multicolor
Yo Aline Multicolor
Shift Blue
Yo Shift Blue
Pinafore Blue
Yo Pinafore Blue
Maxi Maroon
Yo Maxi Maroon
Maxi Red Dress Wit
Yo Maxi Red Dress Wit
Shift White
Yo Shift White
Gathered Black
Yo Gathered Black
Two Piece Dress 

In [105]:
dict_bigram_sorted= sorted(dict_bigram, key=dict_bigram.get, reverse=True)
print(dict_bigram_sorted)

[('Fit', 'Flare'), ('Dark', 'Blue'), ('Aline', 'Multicolor'), ('Skater', 'Multicolor'), ('Maxi', 'Blue'), ('Maxi', 'Multicolor'), ('Aline', 'Black'), ('Flare', 'Black'), ('Flare', 'Multicolor'), ('Flare', 'Dark'), ('Dress', 'Wit'), ('Flare', 'Pink'), ('Maxi', 'Maroon'), ('High', 'Low'), ('Sheath', 'Maroon'), ('White', 'Black'), ('Maxi', 'Black'), ('Flare', 'Blue'), ('Maxi', 'White'), ('Maxi', 'Dark'), ('Aline', 'Blue'), ('Shirt', 'Yellow'), ('Maxi', 'Red'), ('Two', 'Piece'), ('Piece', 'Dress'), ('Flare', 'Green'), ('Aline', 'Red'), ('Maxi', 'Pink'), ('Aline', 'White'), ('Flare', 'Yellow'), ('Maxi', 'Yellow'), ('White', 'Pink'), ('Light', 'Blue'), ('Skater', 'Red'), ('Skater', 'Yellow'), ('Flare', 'Maroon'), ('Flare', 'White'), ('Skater', 'Maroon'), ('Yellow', 'Dress'), ('Bandage', 'Yellow'), ('Dark', 'Green'), ('Aline', 'Yellow'), ('Bodycon', 'Maroon'), ('Gown', 'Maroon'), ('Gathered', 'Black'), ('Dress', 'Black'), ('Flare', 'Red'), ('Shirt', 'Blue'), ('Low', 'Blue'), ('Low', 'Red'), (

In [106]:
row=dict_bigram_sorted[0:5]+dict_bigram_sorted[-5:]
print(row)

[('Fit', 'Flare'), ('Dark', 'Blue'), ('Aline', 'Multicolor'), ('Skater', 'Multicolor'), ('Maxi', 'Blue'), ('Sheath', 'Black'), ('Skater', 'White'), ('Green', 'Yellow'), ('Bodycon', 'Dark'), ('Layered', 'Light')]


In [107]:
res = [] 
for el in row: 
#         sub = el.split(', ')
    sub=" ".join(el)
    res.append(list(sub.split("*"))) 
print(res)

[['Fit Flare'], ['Dark Blue'], ['Aline Multicolor'], ['Skater Multicolor'], ['Maxi Blue'], ['Sheath Black'], ['Skater White'], ['Green Yellow'], ['Bodycon Dark'], ['Layered Light']]


In [108]:
import csv
with open('dress_top_bottom.csv', 'w', newline='') as file:
    res = [] 
    for el in row: 
#         sub = el.split(', ')
        sub=" ".join(el)
        res.append(list(sub.split("*"))) 

    for i in range(len(res)):
        res[i]=res[i]+item_bigram[res[i][0]]

    print(res)
    writer = csv.writer(file)
    writer.writerows(res)

[['Fit Flare', 0, 5, 6, 9, 11, 17, 34, 37, 40, 43, 67, 77, 80, 81, 83, 84, 87, 98, 102, 103, 107, 110, 114, 122, 123, 131, 135, 139, 142, 143, 144, 147, 148, 156, 158, 172, 176, 180, 181, 184, 193, 195, 198, 201, 205, 219, 222, 244, 245, 248, 254, 265, 267, 276, 277, 284, 286, 287, 289, 290, 297, 301, 304, 309, 316, 320, 322, 323, 325, 331, 333], ['Dark Blue', 35, 43, 49, 70, 77, 78, 120, 148, 168, 179, 180, 189, 198, 220, 244, 247, 248, 254, 256, 261, 270, 276, 309, 316, 318, 319], ['Aline Multicolor', 1, 7, 10, 19, 65, 116, 121, 140, 153, 155, 163, 168, 221, 237, 246, 258, 329], ['Skater Multicolor', 8, 28, 30, 38, 42, 44, 57, 64, 74, 105, 170, 178, 283, 288], ['Maxi Blue', 15, 39, 46, 50, 137, 177, 197, 230, 300, 303, 306, 312, 314, 327], ['Sheath Black', 313], ['Skater White', 317], ['Green Yellow', 323], ['Bodycon Dark', 326], ['Layered Light', 335]]


In [162]:
import urllib.request
def download_img(url,file_name,File_path):
    full_path = File_path + file_name + '.jpg'
    urllib.urlretrieve(url,full_path)

download_img('https://rukminim1.flixcart.com/image/832/832/xif0q/shoe/i/g/k/9-chrome-02-asian-slate-blue-navy-original-imaghqfkcmagt7gk.jpeg?q=70',"Okay",'D:/Downloads/GriD_Fashion-master/GriD_Fashion-master/static')
print("completed")

NameError: name 'item' is not defined